# Agriculture Case Study

### Background
During a normal year, sugar cane in Queensland typically flowers early May through June; July to November is typically cane harvesting season.

### The Problem
While sugar is growing, fields may look visually similar, but health or growth rates from these fields can be quite different, leading to variability and unpredictability in revenue. Identifying underperforming crops can have two benefits:

-  Ability to scout for frost or disease damage.
-  Ability to investigate poor performing paddocks and undertake management action such soil testing or targeted fertilising to improve yield.

### Digital Earth Australia Use Case
Satellite Imagery can be used to measure pasture health over time and identify any changes in growth patterns between otherwise similar paddocks.

The Normalised Difference Vegetation Index (NDVI) describes the difference between visible and near-infrared reflectance of vegetation cover. This index estimates the density of green on an area of land and can be used to track the health and growth of sugar as it matures. Comparing the NDVI of two similar paddocks will help to identify any anomalies in growth patterns.

In this example, data from the European Sentinel 2 satellites is used to make a near real-time assessment of crop growing patterns, facillitating management decisions in the field. This data is made available through the Copernicus Regional Data Hub and Digital Earth Australia within 1-2 days of capture.

The worked example below takes users through the code required to:
-  Create a time series data cube over a farming property.
-  Select multiple paddocks for comparison.
-  Create graphs to identify crop performance trends over the previous month.
-  Interpret the results.

### Technical details

Products used: NDVI
Satellite data / Time series: Sentinel 2 / Near real time provision over 30 days

**To begin, let's add our apps to the current notebook. Click on the cell below and either click run from the toolbar above or click control+enter on your keyboard. The following cell also loads a configuration file that allows us to access the necessary data.**

In [ ]:
%matplotlib notebook
import matplotlib as mpl
mpl.rcParams.update({'figure.autolayout': True})

from utils.casestudy_agriculture_functions import load_agriculture_data, run_agriculture_app

In [ ]:
ds_s2_test = load_agriculture_data()

In [ ]:
run_agriculture_app(ds_s2_test)